In [10]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
import bs4
from datetime import datetime
import pymongo
import re


def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


options = webdriver.FirefoxOptions()

# Going undercover:
# option.add_argument("--incognito")
# options.add_argument("--headless")

login_url = 'https://sattacademy.com/login'

start = time.time()

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
driver.get(login_url)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
email_field = wait.until(EC.presence_of_element_located((By.ID, "email")))

# Find the email and password input fields and the login button
email_field = driver.find_element(By.ID, "email")
password_field = driver.find_element(By.ID, "password")
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

# Enter your email and password
email_field.send_keys("ankonbiswas@live.com")
password_field.send_keys("kratos040471")

# Click the login button
login_button.click()

pagination_url = "https://sattacademy.com/job-solution/subject-question/sid=750"
driver.get(pagination_url)
driver.maximize_window()

time.sleep(5)

html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')



test_mode_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn.btn-sm.btn-danger.sa-btn-danger.test-mode")))
driver.execute_script("arguments[0].remove();", test_mode_button)

explaination_buttons = driver.find_elements(By.CLASS_NAME,"description-btn")
j = 0
# explaination_buttons[j].click()

# modal_close_button = driver.find_element(By.ID,"closeLoginModal")
# modal_close_button.click()

while True:
    print(j)
    if j==len(explaination_buttons):
        print('here')
        break
    else:
        #explaination_buttons = driver.find_elements(By.CLASS_NAME,"description-btn")
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView(true);", explaination_buttons[j])
        explaination_buttons[j].click()
        time.sleep(4)
        j+=1

# question_divs = soup.find("span", {"id": "all-question-data"}).find_all("div", {"class": "card card-bordered rounded mb-5"})
# for question_div in question_divs:
#     #print(question_div)
#     question_title = question_div.find("div",{"class": "card-title fw-bold cursor-pointer"}).find("span",{"class": "question-span"})
#     print(question_title.get_text())
    



====== WebDriver manager ======
Driver [C:\Users\USER\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe] found in cache


0
1
2
3
4
5


ElementNotInteractableException: Message: Element <div class="description-btn"> could not be scrolled into view
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:349:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:166:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31
